In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras._tf_keras.keras.utils import to_categorical

2025-01-19 10:37:21.385681: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 10:37:21.388901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 10:37:21.397357: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 10:37:21.413838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737301041.443166  436410 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737301041.44

In [2]:
IMAGE_SIZE = (100, 100) 
BATCH_SIZE = 32
EPOCHS = 28
TRAIN_DIR = "train" 
VALIDATION_SPLIT = 0.2 

In [3]:
try:
    categories = sorted(os.listdir(TRAIN_DIR))  #Listed Categories
    if not categories:
        raise ValueError("Training directory empty")
except FileNotFoundError:
    raise FileNotFoundError(f"Training directory '{TRAIN_DIR}' DNE")

In [4]:
# Function to load in OpenCV instead of Pillow
def load_data_with_cv2(directory, categories, image_size):
    images = []
    labels = []
    for idx, category in enumerate(categories):
        category_path = os.path.join(directory, category)
        if not os.path.isdir(category_path):
            print(f"Warning: '{category_path}' is not a directory. Skipping.")
            continue
        for file in os.listdir(category_path):
            img_path = os.path.join(category_path, file)
            try:
                img = cv2.imread(img_path)  # Load image
                if img is None:
                    print(f"Warning: Could not load image '{img_path}'. Skipping.")
                    continue
                img = cv2.resize(img, image_size)  # Resize to Size set in kernal 2
                img = img / 255.0  # pixel values to 0, 1
                
                img_flattened = img.flatten()
                img_batch = np.expand_dims(img_flattened, axis=0)
                
                images.append(img)
                labels.append(idx)
            except Exception as e:
                print(f"Error loading image '{img_path}': {e}")
    if not images:
        raise ValueError("No images found in the dataset")
    return np.array(images), np.array(labels)

In [5]:
# Load training
train_images, train_labels = load_data_with_cv2(TRAIN_DIR, categories, IMAGE_SIZE)

# Shuffle and split the data manually using NumPy
indices = np.arange(len(train_images))
np.random.seed(42) 
np.random.shuffle(indices)

# Split data
split_idx = int(len(train_images) * (1 - VALIDATION_SPLIT))
train_indices, val_indices = indices[:split_idx], indices[split_idx:]
train_images, val_images = train_images[train_indices], train_images[val_indices]
train_labels, val_labels = train_labels[train_indices], train_labels[val_indices]

train_labels = to_categorical(train_labels, num_classes=len(categories))
val_labels = to_categorical(val_labels, num_classes=len(categories))

# TensorFlow datasets for training and validation
train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data = train_data.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_data = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_data = val_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-01-19 10:37:30.040426: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')  # One output per category
])

/home/odruzgal/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,638,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,732,550 (6.61 MB)

 Trainable params: 1,732,550 (6.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step - accuracy: 0.1504 - loss: 2.2880 - val_accuracy: 0.1348 - val_loss: 1.7918
Epoch 2/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 229ms/step - accuracy: 0.1420 - loss: 1.7915 - val_accuracy: 0.2022 - val_loss: 1.7896
Epoch 3/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 232ms/step - accuracy: 0.2070 - loss: 1.7786 - val_accuracy: 0.2022 - val_loss: 1.7892
Epoch 4/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - accuracy: 0.2141 - loss: 1.7791 - val_accuracy: 0.2022 - val_loss: 1.7888
Epoch 5/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - accuracy: 0.2258 - loss: 1.7787 - val_accuracy: 0.2022 - val_loss: 1.7890
Epoch 6/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.2350 - loss: 1.7750 - val_accuracy: 0.2022 - val_loss: 1.7915
Epoch 7/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 230ms/step - accuracy: 0.2061 - loss: 1.7834 - val_accuracy: 0.2135 - val_loss: 1.7887
Epoch 8/28
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 228ms/step - accuracy: 0.2668 - loss: 1.7694 - val_accuracy: 0.

In [10]:
# Save the trained model
model.save("cat_emotion_model.keras")

print("Model training complete. Saved as 'cat_emotion_model.keras'.")

Model training complete. Saved as 'cat_emotion_model.keras'.
